In [4]:
import os
import sys

from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, RandomizedSearchCV
from lightgbm import LGBMClassifier


import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from typing import Dict

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# import lightgbm

from tqdm.auto import tqdm

import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import warnings
import gc
import os
import re
import sys
from collections import Counter
import subprocess

pd.set_option("display.max_rows", 500)
pd.set_option("display.width", 500)
pd.set_option("display.max_columns", 500)

current_user = os.environ.get('USER')

warnings.filterwarnings("ignore")
np.random.seed(42)
tqdm.pandas()

ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
import yaml

with open('/configs/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

## Load data

In [ ]:
df= pd.read_csv('dataset.csv')
df = shuffle(df,random_state=42)

sym_des = pd.read_csv('symptom_Description.csv')
sym_pre = pd.read_csv('symptom_precaution.csv')
df1 = pd.read_csv('Symptom-severity.csv')

## Processing data

### Main table

In [ ]:
for col in df.columns:
    df[col] = df[col].str.replace('_',' ')

In [ ]:
cols = df.columns
data = df[cols].values.flatten()

s = pd.Series(data)
s = s.str.strip()
s = s.values.reshape(df.shape)

df = pd.DataFrame(s, columns=df.columns)

In [ ]:
df = df.fillna(0)

In [ ]:
df1['Symptom'] = df1['Symptom'].str.replace('_',' ')

In [ ]:
vals = df.values
symptoms = df1['Symptom'].unique()

for i in range(len(symptoms)):
    vals[vals == symptoms[i]] = df1[df1['Symptom'] == symptoms[i]]['weight'].values[0]
    
d = pd.DataFrame(vals, columns=cols)

d = d.replace('dischromic  patches', 0)
d = d.replace('spotting  urination',0)
df = d.replace('foul smell of urine',0)

## Train LightGBM

### Model hyperparameters

In [6]:
lgb_params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "max_bin": 5,
        "max_depth" : 2,
        "num_leaves": 5,
        "min_data_in_leaf" : 5,
        "learning_rate": 0.2,
        "bagging_fraction": 0.7,
        "feature_fraction": 0.5,
        "bagging_seed": 2018,
        "verbosity": -1
    }

### Train

In [ ]:
data = df.iloc[:,1:].values
labels = df['Disease'].values

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, train_size = 0.8,random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
full_lgb_model = mf.LGBModel(lgb_params, test_size=0.1)
full_lgb_model.fit(data, labels)


rnd_forest = RandomForestClassifier(random_state=42, max_features='sqrt', n_estimators= 500, max_depth=13)
rnd_forest.fit(x_train,y_train)
preds=rnd_forest.predict(x_test)
print(x_test[0])
print(preds[0])
conf_mat = confusion_matrix(y_test, preds)
df_cm = pd.DataFrame(conf_mat, index=df['Disease'].unique(), columns=df['Disease'].unique())
print('F1-score% =', f1_score(y_test, preds, average='macro')*100, '|', 'Accuracy% =', accuracy_score(y_test, preds)*100)